# Permutation engine examples

This shows how to use the permutation engine to permute gates of a certain type towards the beginning or end of the circuit.
It relies on the implementation of permutation rules between different gates. Currently, only rotational gates + Pauli + S + H + T and TimeEvolution gates are supported. So far there is no support for Controlled gates. But a CNOT can easily decomposed into two pi/4 rotations.

In [6]:
import projectq
import cmath
from projectq.backends import CommandPrinter, Simulator
from projectq.ops import *
from projectq.cengines import PermutePi4Front

#### First generate the main engine and the order of all the engines in the compiler stack.

In [7]:
engines = [CommandPrinter(),projectq.cengines.TagRemover(),PermutePi4Front()]
eng = projectq.MainEngine(backend=CommandPrinter(), engine_list=engines)

Now lets generate a very simple circuit

In [8]:
qubit = eng.allocate_qubit()
X | qubit
Y | qubit
X | qubit
T | qubit
T | qubit
eng.flush()

Allocate | Qureg[0]
Deallocate | Qureg[0]
X | Qureg[0]
Y | Qureg[0]
X | Qureg[0]
T | Qureg[0]
T | Qureg[0]
Allocate | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rz(11.780972450962) | Qureg[0]
Rx(3.14159265359) | Qureg[0]
Ry(3.14159265359) | Qureg[0]
Rx(3.14159265359) | Qureg[0]


Now lets take a look at two qubit gates. Such as the CNOT gate. For this we need to use the replacement rules provided by the projectq replacement engine. For this we first need to define which gates are allowed and which need to be replaced.

In [9]:
from projectq.setups import restrictedgateset

In [12]:
engines2 = restrictedgateset.get_engine_list(one_qubit_gates=(X,Y,Z,Rx,Ry,Rz,H,T,S),
                                                two_qubit_gates=(),
                                                other_gates=(TimeEvolution,))
print(engines2)

[<projectq.cengines._replacer._replacer.AutoReplacer object at 0x7f11ba3005f8>, <projectq.cengines._tagremover.TagRemover object at 0x7f11ba300ba8>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7f11ba300be0>, <projectq.cengines._optimize.LocalOptimizer object at 0x7f11ba300c18>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7f11ba300c50>, <projectq.cengines._tagremover.TagRemover object at 0x7f11ba300c88>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7f11ba300cc0>, <projectq.cengines._optimize.LocalOptimizer object at 0x7f11ba300cf8>, <projectq.cengines._replacer._replacer.AutoReplacer object at 0x7f11ba300d30>, <projectq.cengines._tagremover.TagRemover object at 0x7f11ba300d68>, <projectq.cengines._replacer._replacer.InstructionFilter object at 0x7f11ba300da0>, <projectq.cengines._optimize.LocalOptimizer object at 0x7f11ba300dd8>]


It seems there is no decomposition of CNOTs into multi qubit rotation gates. This we still need to implement.

In [14]:
eng2 = projectq.MainEngine(backend=CommandPrinter(), engine_list=engines2)

qubit1 = eng2.allocate_qubit()
qubit2 = eng2.allocate_qubit()

H | qubit1
CNOT | (qubit1, qubit2)
eng2.flush()

Deallocate | Qureg[1]
Deallocate | Qureg[2]
Allocate | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) | Qureg[1]
H | Qureg[1]
Ry(7.853981633974) |

AttributeError: 'numpy.ndarray' object has no attribute '_collapse'
 raised in:
'  File "/usr/local/lib/python3.6/dist-packages/numpy/matrixlib/defmatrix.py", line 606, in all'
'    return N.ndarray.all(self, axis, out, keepdims=True)._collapse(axis)'

In [16]:
import projectq.setups.decompositions
rule_set = projectq.cengines.DecompositionRuleSet(modules=[projectq.setups.decompositions])

In [17]:
projectq.setups.decompositions

<module 'projectq.setups.decompositions' from '/mnt/c/Users/danie/Documents/PhD/codes/QC_benchmark/projectq/setups/decompositions/__init__.py'>